In [1]:
#!pip install requests2
#!pip install beautifulsoup4
#!pip install selenium

In [2]:
#import time
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
from bs4 import BeautifulSoup
import json
import urllib.parse

In [3]:
url = 'https://newsapi.org/v2/everything?'
api_key = '126614c036da4f5ba425e28b9b04b12b'
busca = 'bolsonaro'

In [4]:
keywords = ["bolsonaro", "facada"]

parametros = {
    'q' : ' AND '.join(keywords),
    'pageSize' : 6,
    'apiKey' : api_key,
    'language' : 'pt'
}

In [5]:
#chamando a news api
resp = requests.get(url, params = parametros)
print(resp)
resp_json = resp.json()

#resp_json

<Response [200]>


In [6]:
#pegando a parte da resposta que interessa
print(resp_json.keys())
resp_articles = resp_json['articles']
len(resp_articles)

dict_keys(['status', 'totalResults', 'articles'])


6

In [7]:
#transformando os artigos de json pra df
data = json.loads(json.dumps(resp_articles))
dfArtigos = json_normalize(data)
type(dfArtigos)
dfArtigos

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,Fábio Py,Cristofascismo em 7 atos: como Bolsonaro usou ...,"Em plena Páscoa, Bolsonaro desenvolveu uma con...",https://theintercept.com/2020/05/01/cristofasc...,https://theintercept.com/wp-uploads/sites/1/20...,2020-05-01T04:03:49Z,Foto: Andressa Anholete/Getty Images\r\nAo per...,None,Theintercept.com
1,https://www.facebook.com/bbcnews,"'E daí?': quem é Alexandre Ramagem, o amigo da...","Delegado federal, que dirigia Agência Brasilei...",https://www.bbc.com/portuguese/brasil-52453022,https://ichef.bbci.co.uk/news/1024/branded_por...,2020-04-28T10:45:33Z,Direito de imagemCarolina Antunes/PRImage capt...,None,Bbc.com
2,tbronzatto,Por que Bolsonaro quer trocar o comando da Pol...,"Investigações que atingiram aliados, inquérito...",https://veja.abril.com.br/brasil/por-que-bolso...,https://abrilveja.files.wordpress.com/2019/12/...,2020-04-23T20:20:30Z,O presidente Jair Bolsonaro se elegeu hasteand...,None,Abril.com.br
3,Rafael Neves,Lava Jato suspeitava que Alexandre Ramagem era...,Escolhido por Bolsonaro como novo chefe da Pol...,https://theintercept.com/2020/04/27/vaza-jato-...,https://theintercept.com/wp-uploads/sites/1/20...,2020-04-28T01:51:12Z,O delegado Alexandre Ramagem\r\n ainda nem ass...,None,Theintercept.com
4,https://www.facebook.com/bbcnews,Da negação da demissão à facada de Adélio: com...,"Apoiadores do presidente no WhatsApp, Facebook...",https://www.bbc.com/portuguese/brasil-52444072,https://ichef.bbci.co.uk/news/1024/branded_por...,2020-04-28T14:13:38Z,Direito de imagemREUTERS/Ueslei MarcelinoImage...,None,Bbc.com
5,None,"Em live de Páscoa, Bolsonaro relaciona ressurr...","Neste domingo, 12, o presidente Jair Bolsonaro...",https://catracalivre.com.br/cidadania/em-live-...,https://catracalivre.com.br/wp-content/uploads...,2020-04-12T22:50:59Z,"Neste domingo, 12, o presidente Jair Bolsonaro...",None,Catracalivre.com.br


In [8]:
dfConteudo = pd.DataFrame(columns=['nr_noticia', 'texto', 'tamanho', 'origem'])
contador = 0

for url in dfArtigos['url']:
    print(url)
    article = requests.get(url)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('body')
    p = body[0].find_all('p')
    for y in p:
        text = y.get_text().strip()
        if(len(text) > 0):
            dfConteudo = dfConteudo.append({"nr_noticia":contador, "texto":text, "tamanho":len(text), "origem":url}, ignore_index=True)
    
    contador = contador + 1

print('concluido')

https://theintercept.com/2020/05/01/cristofascismo-bolsonaro-pascoa/
https://www.bbc.com/portuguese/brasil-52453022
https://veja.abril.com.br/brasil/por-que-bolsonaro-quer-trocar-o-comando-da-policia-federal/
https://theintercept.com/2020/04/27/vaza-jato-deltan-desconfiava-ramagem/
https://www.bbc.com/portuguese/brasil-52444072
https://catracalivre.com.br/cidadania/em-live-de-pascoa-bolsonaro-relaciona-ressurreicao-a-facada-de-2018/
concluido


In [17]:
noticia = dfConteudo[dfConteudo['nr_noticia']==2] 

print(noticia)
noticiaFiltrado = noticia[noticia['tamanho'] > 50]

textoNoticia = ""
for nf in noticiaFiltrado['texto']:
    textoNoticia = textoNoticia + "\n" + nf

    ##############LEMBRAR DE PEGAR TITULOS TAMBEM
print('\nqtd caracteres na noticia: ' + str(len(textoNoticia)))
print(textoNoticia)

    nr_noticia                                              texto tamanho  \
92           2  O presidente Jair Bolsonaro se elegeu hasteand...     519   
93           2  Mas por que, afinal, Bolsonaro quer trocar a c...     110   
94           2  Desde setembro do ano passado, o presidente es...     302   
95           2  No mesmo mês, a Polícia Federal deflagrou uma ...     272   
96           2  Outro ponto que desgastou Valeixo foi a invest...     508   
97           2  Além disso, aliados do presidente passaram a d...     628   
98           2  Em janeiro, Valeixo teve uma conversa com Moro...     401   
99           2  Nos últimos dias, o diretor-geral da PF começo...     536   
100          2  Em meio a essa confusão, Valeixo ainda não foi...     271   
101          2                            A partir de R$ 9,90/mês      23   
102          2                            A partir de R$ 6,90/mês      23   
103          2                            A partir de R$ 4,90/mês      23   

In [14]:
import spacy
nlp = spacy.load("pt_core_news_sm")

In [18]:
doc = nlp(textoNoticia)
doc.ents

(Jair Bolsonaro,
 Brasília,
 Operação Lava-Jato,
 Sergio Moro,
 Justiça,
 Polícia Federal,
 Bolsonaro,
 Moro,
 PF,
 Maurício Valeixo,
 Curitiba,
 Lula,
 Bolsonaro,
 Polícia Federal,
 Lava-Jato?,
 Valeixo,
 Bolsonaro,
 Hélio Lopes,
 Hélio Negão,
 PF,
 Polícia Federal,
 Fernando Bezerra Coelho,
 MDB-PE,
 Senado,
 Congresso,
 Bolsonaro,
 Moro,
 Valeixo,
 Adélio Bispo de Oliveira,
 Bolsonaro,
 PF,
 Juiz de Fora,
 Minas Gerais,
 Polícia Federal,
 Palácio do Planalto,
 Valeixo,
 Moro,
 PF,
 Alberto Fraga,
 Bolsonaro,
 Segurança do Distrito Federal,
 Anderson Torres,
 Agência Brasileira de Inteligência,
 Abin,
 Alexandre Ramagem,
 Bolsonaro,
 Valeixo,
 Moro,
 Desgastado,
 em Portugal,
 Justiça,
 PF,
 Lava-Jato,
 PF,
 Valeixo,
 Bolsonaro,
 Moro,
 Polícia Federal,
 Justiça,
 Brasília,
 Valeixo,
 Bolsonaro,
 Polícia Federal,
 Copyright)